In [1]:
# Necessary imports
import nltk
import glob
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
pos_files = glob.glob("data/clean_positive_train_0.csv")
neg_files = glob.glob("data/clean_negative_train_0.csv")

df_pos_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in pos_files[:3]]
df_neg_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in neg_files[:3]]

Now we make features for each dataset that we have, calculating all these features takes a long long time...

In [9]:
df = [pd.concat([df_pos, df_neg]) for (df_pos, df_neg) in zip(df_pos_list, df_neg_list)]

# Now we have scrambeled dataframes of 20000 entries, with features such as binary scores and profanity
df[0].describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04
mean,0.500000,1127.415600,0.000050,0.973000,1347.648050,0.000350,0.033150,0.197250,2.679215e-01
std,0.500013,1375.725312,0.007071,0.162087,1465.051101,0.018705,0.283466,0.397933,2.922975e-01
min,0.000000,-2946.000000,0.000000,0.000000,-8907.000000,0.000000,-1.000000,0.000000,1.178028e-22
25%,0.000000,-129.000000,0.000000,1.000000,117.000000,0.000000,-0.037500,0.000000,8.408572e-02
50%,0.500000,820.000000,0.000000,1.000000,783.000000,0.000000,0.000000,0.000000,1.238206e-01
75%,1.000000,2183.000000,0.000000,1.000000,2329.000000,0.000000,0.160000,0.000000,3.362519e-01
max,1.000000,9582.000000,1.000000,1.000000,14097.000000,1.000000,1.000000,1.000000,1.000000e+00


In [10]:
df[0].head()

,text,score,ups,controversiality,parent_text,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
0,twitter try comment article current activities,1,9582,0,fucking faggot,0,-7526,0,0.000000,0,0.121710
1,well exactly sounds like shoebox least wheneve...,1,9531,0,elaborate cum box please,1,3841,0,0.010472,1,0.999996
2,soviet russia bomb disarms,1,8545,0,dont live russia anymore going back time soon ...,1,621,0,0.000000,0,0.084165
3,runin senitur yolo,1,7430,0,made realize future presidents probably facebo...,1,4651,0,0.000000,0,0.120956
4,step motherfucker,1,7173,0,sex step mom dad around junior high school goi...,1,4251,0,0.000000,1,0.658445


In [41]:
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)
# Create the word_index list based on all our data",
text_data = [np.array2string(df[single_df]['text'].values.astype(str)) for single_df in range(len(df))]
text_data = ' '.join(text_data)
tokenizer.fit_on_texts(text_data)

In [ ]:
feature_data = [np.array([])]
score_data = [np.array([])]

X = []
Y = []

for n in range(len(df)):
    cur_df = df[n]
    df[n].dropna(axis=0, inplace=True)
    size = len(cur_df)
    for index, row in cur_df.iterrows():
        sentiment = row['sentiment']
        profanity = row['profanity_prob']
        features = np.hstack((sentiment, profanity))
        if index == 0:
            feature_data[n] = features
            score_data[n] = row['score']
        else:
            feature_data[n] = np.vstack([feature_data[n], features])
            score_data[n] = np.vstack([score_data[n], row['score']])
        

print(feature_data[0].shape)
print(score_data[0].shape)

In [ ]:
X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(feature_data[n], text_score[n], test_size=0.20, random_state=42)
X_train.append(X_train_tmp)
X_test.append(X_test_tmp)
y_train.append(y_train_tmp)
y_test.append(y_test_tmp)

X_train[n] = tf.keras.utils.normalize(X_train[n], axis=1)
X_test[n] = tf.keras.utils.normalize(X_test[n], axis=1)

## Finally we can do some neural networks!

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(24, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(16, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
train_data = X_train[0]

test_data = X_test[0]

history = model.fit(train_data,
                    y_train[0],
                    epochs=40,
                    validation_data=(test_data, y_test[0]),
                    verbose=1)